### **YELP DATASET PROJECT**


#### **Import**

In [1]:
import pandas as pd
import os
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from chart_studio.plotly import plot, iplot


import dash
from dash import Dash, html, dcc

#### Data collection

In [2]:
for dirname, _, filenames in os.walk('data/yelp dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data/yelp dataset\yelp_academic_dataset_business.json
data/yelp dataset\yelp_academic_dataset_checkin.json
data/yelp dataset\yelp_academic_dataset_review.json
data/yelp dataset\yelp_academic_dataset_tip.json
data/yelp dataset\yelp_academic_dataset_user.json


In [3]:
def load_data(name):

    data_path = r'data/yelp dataset/yelp_academic_dataset_{}.json'.format(name)
    load = pd.read_json(data_path, lines=True)
    return load

In [4]:
business_df = load_data('business')

#### Data cleaning

In [ ]:
business_df = business_df.loc[business_df['is_open']==1] # dropping closed business
business_df = business_df[business_df['categories'].notnull()] # dropping bussiness without categories

#### Data processing

In [ ]:
# creating list of category
category_df = business_df[['categories']]
category_df = category_df['categories'].str.split(',', expand=True)
#category_df = category_df.drop_duplicates()

In [ ]:
# join all categories to one list
cate_temp = []
for i in range(0,35):
    temp = list(category_df[i])
    cate_temp += temp

In [ ]:
category = pd.DataFrame(cate_temp, columns=['category'])
category = category.dropna()
category['category'] = category['category'].str.strip()
# category = category.drop_duplicates()
# list(category['category'])

In [ ]:
cate_count = category.value_counts().reset_index(name='count')
top10_cate_count = cate_count.iloc[:10]
top10_cate = px.bar(top10_cate_count, y='category', x='count', text_auto=True,
             title='Top 10 bussiness category')
top10_cate.update_layout(autosize=True,
    yaxis=dict(autorange='reversed'))

In [ ]:
res_lis = 'Restaurants|Bars|Desserts|Coffee & Tea|Donuts|Food'
restaurant_df = business_df.loc[business_df['categories'].str.contains(res_lis)] # selecting restaurants
restaurant_df = restaurant_df.loc[restaurant_df['categories'].str.contains('Grocery') == False] # dropping grocery 

# categorical data
rest_cate = restaurant_df[['categories']]
rest_cate = rest_cate['categories'].str.split(',', expand=True)

cate_temp = []
for i in range(0,26):
    temp = list(rest_cate[i])
    cate_temp += temp

In [ ]:
rest_cate = pd.DataFrame(cate_temp, columns=['category'])
rest_cate['category'] = rest_cate['category'].str.strip()
rest_cate['category'] = rest_cate.loc[(rest_cate['category']!='Restaurants') & (rest_cate['category']!='Food')]
rest_cate = rest_cate.dropna()

In [ ]:
rest_cate_count = rest_cate.value_counts().reset_index(name='count')
top10_rest_cate_count = rest_cate_count.iloc[:10]
top_10_rest_cate = px.bar(top10_rest_cate_count, y='category', x='count', text_auto=True,
             title='Top 10 restaurants category')
top_10_rest_cate.update_layout(yaxis=dict(autorange='reversed'))
top_10_rest_cate.update_layout(
    autosize=True)

In [ ]:
# group by state
state_gp = restaurant_df.groupby(['state'])
count_res = state_gp['business_id'].count().sort_values(ascending=False).reset_index(name='count')
count_res = count_res.loc[count_res['count']>2]
count_res_plot = px.bar(count_res, x='state', y='count', text_auto=True, title='Number of restaurants in each state')
count_res_plot.update_layout(
    autosize=True)

In [ ]:
city_gp = restaurant_df.groupby(['state', 'city'])
count_res_city = city_gp['name'].count().reset_index(name='count_res').sort_values(by='count_res', ascending=False)
count_res_city

,state,city,count_res
786,PA,Philadelphia,4308
161,FL,Tampa,2434
267,IN,Indianapolis,2320
909,TN,Nashville,2088
23,AZ,Tucson,2048
...,...,...,...
146,FL,St Pete,1
280,IN,Plainfiled,1
148,FL,St Petersberg,1
711,PA,Hollywood,1


In [ ]:
# rating 
res_name_gp = restaurant_df.groupby(by='name')
avg_rating = res_name_gp['stars'].mean().reset_index(name='rating')
count_res = res_name_gp['name'].count().reset_index(name='count_res')
count_review = res_name_gp['review_count'].sum().reset_index(name='count_review')

avg_rating['count_res'] = count_res['count_res']
avg_rating['count_review'] = count_review['count_review']

avg_rating = avg_rating.sort_values(by='count_review', ascending=False)
top10_res = avg_rating[:10]

top10_res_plot = px.bar(top10_res, x='rating', y='name', 
             text_auto=True, title='Average rating for top 10 restaurant on reviews',
             hover_data=['count_res', 'count_review'])

top10_res_plot.update_layout(yaxis=dict(autorange='reversed'))

top10_res_plot.update_layout(
    autosize=True
)

top10_res_plot.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    )
)

top10_res_plot.show()

In [ ]:
top10_res

,name,rating,count_res,count_review
22924,Starbucks,3.102479,605,18224
15507,McDonald's,1.868702,655,16490
7381,Dunkin',2.302395,501,9681
5115,Chipotle Mexican Grill,2.381757,148,9071
8618,First Watch,3.896552,58,8688
463,Acme Oyster House,4.000000,2,8343
23737,Taco Bell,2.141813,342,8017
4901,Chick-fil-A,3.381410,156,7943
17504,Oceana Grill,4.000000,1,7400
3516,Buffalo Wild Wings,2.347458,59,6810


In [ ]:
# cloer look at PA and FL
PA_rest_df = restaurant_df.loc[restaurant_df['state'] == 'PA']
PA_city_gp = PA_rest_df.groupby(by='city')
count_rest_PA_city = PA_city_gp['business_id'].count().sort_values(ascending=False).reset_index(name='count_rest')
PA_rest_gp = PA_rest_df.groupby(by='name')
PA_avg_rating = PA_rest_gp['stars'].mean().reset_index(name='avg_rating')
PA_review_count = PA_rest_gp['review_count'].sum().reset_index(name='count_review')
PA_count_rest = PA_rest_gp['name'].count().reset_index(name='count_res')

PA_avg_rating['count_review'] = PA_review_count['count_review']
PA_avg_rating['count_rest'] = PA_count_rest['count_res']

top10_rest_count = PA_avg_rating.sort_values(by='count_rest', ascending=False)[:10]
top10_rest_review = PA_avg_rating.sort_values(by='count_review', ascending=False)[:10]

In [ ]:
restaurant_plot = restaurant_df[['name', 'state', 'review_count', 'stars']]

In [ ]:
def top_10_rest(state_name):
    
    s_rest_df = restaurant_plot.loc[restaurant_plot['state'] == state_name]
    s_rest_gp = s_rest_df.groupby(by='name')
    s_avg_rating = s_rest_gp['stars'].mean().reset_index(name='avg_rating')
    s_review_count = s_rest_gp['review_count'].sum().reset_index(name='count_review')
    s_count_rest = s_rest_gp['name'].count().reset_index(name='count_res')

    s_avg_rating['count_review'] = s_review_count['count_review']
    s_avg_rating['count_rest'] = s_count_rest['count_res']

    top10_rest_review = s_avg_rating.sort_values(by='count_review', ascending=False)[:10]

    return top10_rest_review

In [ ]:
top_10_rest('FL')

,name,avg_rating,count_review,count_rest
1015,Columbia Restaurant,4.000000,3877,2
1181,Datz,3.750000,3445,2
3933,Starbucks,3.230000,3203,100
2698,McDonald's,1.932773,3104,119
1499,First Watch,3.880952,3086,21
4719,Ulele,4.000000,3064,1
1317,Dunkin',2.064516,3036,124
425,Bern's Steak House,4.500000,2924,1
1594,Frenchy's Rockaway Grill,4.000000,2301,1
2940,Noble Crust,4.333333,2258,3


In [ ]:
top10_rest_count

,name,avg_rating,count_review,count_rest
1842,Dunkin',2.377404,2894,208
7219,Wawa,3.272109,2073,147
6003,Starbucks,2.976852,3121,108
4028,McDonald's,1.793269,2339,104
953,CVS Pharmacy,2.326316,1243,95
7237,Wendy's,1.953125,1332,64
1766,Domino's Pizza,1.916667,1392,54
5457,Rite Aid,2.500000,505,49
5453,Rita's Italian Ice,3.630435,568,46
915,Burger King,2.034091,724,44


In [ ]:
fig = px.bar(top10_rest_count, x='avg_rating', y='name', 
             text_auto=True, title='Average rating for top 10 restaurant on number of restaurnat in PA',
             hover_data=['count_review', 'count_rest'])

fig.update_layout(yaxis=dict(autorange='reversed'))

fig.update_layout(
    autosize=False,
    width=700,
    height=500
)

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    )
)

fig.show()

In [ ]:
pa_rest_plot = px.bar(top10_rest_review, x='avg_rating', y='name', 
             text_auto=True, title='Average rating for top 10 restaurant in PA on number of review' ,
             hover_data=['count_review', 'count_rest'])

pa_rest_plot.update_layout(yaxis=dict(autorange='reversed'))

pa_rest_plot.update_layout(
    autosize=False,
    width=700,
    height=500
)

pa_rest_plot.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    )
)

In [ ]:
# # save visulization data to csv
# top10_cate_count.to_csv(r'./visuization/top_10_cate.csv')
# top10_rest_cate_count.to_csv(r'./visuization/top_10_rest_cate.csv')
#count_res.to_csv(r'./visulization/data/count_rest_state.csv')
#top10_res.to_csv(r'./visulization/data/top10_rest.csv')
# restaurant_plot.to_csv(r'./visulization/data/restaurant_plot.csv')

In [ ]:
count_res

,name,count_res
0,#1 Mongolian BBQ - Best Stir Fried Noodles In ...,1
1,$5 Fresh Burger Stop,1
2,&pizza - UPenn,1
3,&pizza - Walnut,1
4,&pizza - Willow Grove,1
...,...,...
28946,ÀLAVITA,1
28947,Àrdana Food & Drink,1
28948,Éclat Chocolate,1
28949,éclair délicieux,1


In [ ]:
fig = px.bar(top10_rest_review, x='avg_rating', y='name', 
             text_auto=True, title='Average rating for top 10 restaurant in PA on number of review' ,
             hover_data=['count_review', 'count_rest'])

fig.update_layout(yaxis=dict(autorange='reversed'))

fig.update_layout(
    autosize=False,
    width=700,
    height=500
)

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    )
)

fig.show()

#### User dataset

In [3]:
user_df = load_data('user')

In [4]:
user_df['yelping_since'] = pd.to_datetime(user_df['yelping_since'])
user_df['year'] = user_df['yelping_since'].dt.year
user_df['month'] = user_df['yelping_since'].dt.month

user_df['like'] = user_df['useful'] + user_df['funny'] + user_df['cool']

In [ ]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 25 columns):
 #   Column              Dtype         
---  ------              -----         
 0   user_id             object        
 1   name                object        
 2   review_count        int64         
 3   yelping_since       datetime64[ns]
 4   useful              int64         
 5   funny               int64         
 6   cool                int64         
 7   elite               object        
 8   friends             object        
 9   fans                int64         
 10  average_stars       float64       
 11  compliment_hot      int64         
 12  compliment_more     int64         
 13  compliment_profile  int64         
 14  compliment_cute     int64         
 15  compliment_list     int64         
 16  compliment_note     int64         
 17  compliment_plain    int64         
 18  compliment_cool     int64         
 19  compliment_funny    int64         
 20  co

In [ ]:
user_df[['review_count', 'like', 'fans']].describe()

,review_count,like,fans
count,1.987897e+06,1.987897e+06,1.987897e+06
mean,2.339441e+01,8.305979e+01,1.465740e+00
std,8.256699e+01,1.600320e+03,1.813075e+01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,1.000000e+00,0.000000e+00
50%,5.000000e+00,4.000000e+00,0.000000e+00
75%,1.700000e+01,1.800000e+01,0.000000e+00
max,1.747300e+04,5.879330e+05,1.249700e+04


In [5]:
sel_user_df = user_df[['user_id', 'name','review_count','average_stars','year', 'month', 'fans','like']].copy()

In [ ]:
sel_user_df.to_csv(r'./visulization/data/sel_user_df.csv')

In [28]:
rating_hisg = px.histogram(sel_user_df, x='average_stars', nbins=20, title='Average rating histogram')

In [27]:
user_join_his = px.histogram(sel_user_df, x='year', title='When does Yelp become popular?')